In [10]:
import psycopg2
import xml.sax
from xml.sax.handler import ContentHandler
import pandas as pd

In [11]:
article = []
inproceedings = []
author = []

In [12]:
class dbHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.Current_Data = ""
        self.pubkey = ""
        self.title = ""
        self.journal = ""
        self.year = ""
        self.booktitle = ""
        self.author_set = set()
        self.a = []
        
    def startElement(self, tag, attributes):
        self.Current_Data = tag
        if tag == "article":
            self.pubkey = attributes["key"]
            self.title = ""
            self.journal = ""
            self.year = ""
            self.booktitle = ""
            self.author_set = set()
            self.a = []
            
            
        if tag == "inproceedings":
            self.pubkey = attributes["key"]
            self.title = ""
            self.journal = ""
            self.year = ""
            self.booktitle = ""
            self.author_set = set()
            self.a = []
        if tag == "author":
            self.a = []
    
            
            
    def endElement(self, tag):
        if tag == "article":
            article.append((self.pubkey, self.title, self.journal, self.year))
            for x in self.author_set:
                author.append((self.pubkey, x))
        if tag == "inproceedings":
            inproceedings.append((self.pubkey, self.title, self.booktitle, self.year))
            for x in self.author_set:
                author.append((self.pubkey, x))
        if tag == "author":
            self.author_set.add(''.join(self.a))
        self.Current_Data == ""
            
        
    def characters(self, content):
        if content == '\n':
            return
        if self.Current_Data == "title":
            self.title = content
        if self.Current_Data == "journal":
            self.journal = content
        if self.Current_Data == "year":
            self.year = content
        if self.Current_Data == "booktitle":
            self.booktitle = content
        if self.Current_Data == "author":
            self.a.append(content)


In [13]:
if ( __name__ == "__main__"):
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)
    Handler = dbHandler()
    parser.setContentHandler( Handler )
    parser.parse("dblp-2022-01-01.xml")

In [14]:
for idx, (q, w, e, r) in enumerate(article):
    if r == '':
        article[idx] = (q, w, e, None)
    else:
        article[idx] = (q, w, e, int(r))

In [15]:
for idx, (q, w, e, r) in enumerate(inproceedings):
    if r == '':
        inproceedings[idx] = (q, w, e, None)
    else:
        inproceedings[idx] = (q, w, e, int(r))

In [21]:
conn = psycopg2.connect(host="127.0.0.1", database="dblp", user="postgres", password="bcbcbc1997126", port="1234")

cur = conn.cursor()
            
for i in article:
    cur.execute("INSERT INTO public.\"article\"(pubkey, title, journal, year) VALUES (%s, %s, %s, %s)", i)
for i in inproceedings:
    cur.execute("INSERT INTO public.\"inproceedings\"(pubkey, title, booktitle, year) VALUES (%s, %s, %s, %s)", i)
for i in author:
    cur.execute("INSERT INTO public.\"Authorship\"(pubkey, author) VALUES (%s, %s)", i)


conn.commit()
conn.close()